In [6]:
import pandas as pd
import nibabel as nib
from glob import glob
import numpy as np
import statsmodels.api as sp
#imported the full paths since it wasn't working otherwise - not sure why 
from statsmodels.stats.weightstats import ttest_ind
from statsmodels.stats.multitest import fdrcorrection


In [23]:
# read in df_tabular with regional and network based data 

df_tabular = pd.read_csv('results/analysis/df_tabular.csv',sep='\t')


In [24]:
#just get LOBE dataset 
df_LOBE = df_tabular[df_tabular["dataset"] == "LOBE"]

In [27]:
df_LOBE

,participant_id,dataset,age,sex,age_onset,ep_duration,pnes,focal_seizure_type,generalized_seizures,dre,...,NFC_Limbic,NFC_Cont,NFC_Default,NSC_Vis,NSC_SomMot,NSC_DorsAttn,NSC_SalVentAttn,NSC_Limbic,NSC_Cont,NSC_Default
349,sub-006,LOBE,9.0,M,8.0,1.0,0.0,motor,1.0,NaN,...,0.242738,0.337645,0.289935,4.158078,4.446592,7.468249,7.537788,6.113632,7.097794,7.533409
350,sub-007,LOBE,9.0,F,7.0,2.0,0.0,motor,0.0,1.0,...,0.039989,0.055922,0.013673,4.174113,4.580525,7.659050,7.325782,6.373010,7.850586,8.161856
351,sub-009,LOBE,8.0,M,8.0,0.0,0.0,motor,0.0,1.0,...,0.147737,0.221310,0.161172,4.737190,4.914379,8.178362,7.942970,6.717391,7.934972,8.000003
352,sub-010,LOBE,15.0,M,7.0,8.0,0.0,non-motor,0.0,0.0,...,0.090205,0.100796,0.137763,4.866766,4.790801,8.518090,7.772193,6.823215,7.446570,7.673076
353,sub-012,LOBE,14.0,F,4.0,10.0,0.0,non-motor,1.0,NaN,...,0.100053,0.078775,0.079218,4.437935,5.252311,8.316320,7.973676,5.279316,7.798454,7.919612
354,sub-013,LOBE,15.0,F,13.0,2.0,0.0,non-motor,0.0,NaN,...,0.098577,0.111910,-0.060181,4.385528,4.652948,8.115841,8.079767,6.332022,6.787390,8.240192
355,sub-014,LOBE,14.0,M,9.0,5.0,0.0,non-motor,1.0,NaN,...,0.163124,0.184982,0.137038,3.913295,4.390988,7.457189,7.216438,5.839379,6.931214,6.931229
356,sub-015,LOBE,10.0,M,7.0,3.0,0.0,motor,NaN,NaN,...,0.088092,0.083745,0.146137,4.796880,4.806006,8.220487,8.515590,6.897235,7.125103,8.940187
357,sub-017,LOBE,15.0,F,10.0,5.0,0.0,non-motor,1.0,1.0,...,0.139618,0.212389,0.068701,4.347981,4.867421,8.900930,7.741240,6.696177,8.214334,8.586060
358,sub-019,LOBE,17.0,F,16.0,1.0,0.0,non-motor,1.0,NaN,...,0.097342,0.059264,0.031013,4.358916,4.771967,8.550381,8.033343,6.900780,8.004336,8.213125


In [294]:
#get temporal and nontemporal groups
temporal = df_LOBE[df_LOBE["loc_eeg_temporal"] == 1]
nontemporal = df_LOBE[df_LOBE["loc_eeg_temporal"] != 1]

#make sure pulling right subs
df_temporal = temporal["participant_id"]
df_nontemporal = nontemporal["participant_id"]

print(df_temporal)

350    sub-007
351    sub-009
353    sub-012
354    sub-013
355    sub-014
356    sub-015
358    sub-019
363    sub-025
364    sub-026
369    sub-035
Name: participant_id, dtype: object


In [87]:
x1 = temporal["NSFC_DorsAttn"]
x2 = nontemporal["NSFC_DorsAttn"]

In [88]:
tstats,pvals,dfs = ttest_ind(x1, x2, alternative='two-sided', usevar='pooled', weights=(None, None), value=0)
pvals

0.018448967504940048

In [90]:
#get motor and nonmotor groups
motor = df_LOBE[df_LOBE["focal_seizure_type"] == "motor"]
nonmotor = df_LOBE[df_LOBE["focal_seizure_type"] != "motor"]

#make sure pulling right subs
df_motor = motor["participant_id"]
df_nonmotor = nonmotor["participant_id"]

print(df_nonmotor)

352    sub-010
353    sub-012
354    sub-013
355    sub-014
357    sub-017
358    sub-019
359    sub-020
360    sub-021
362    sub-024
363    sub-025
365    sub-027
368    sub-033
Name: participant_id, dtype: object


In [133]:
x1 = motor["NSC_SomMot"]
x2 = nonmotor["NSC_SomMot"]

In [134]:
tstats,pvals,dfs = ttest_ind(x1, x2, alternative='two-sided', usevar='pooled', weights=(None, None), value=0)
pvals

0.7746711076604857

In [137]:
#get dre and nondre groups
dre = df_LOBE[df_LOBE["dre"] == 1]
nondre = df_LOBE[df_LOBE["dre"] != 1]

#make sure pulling right subs
df_dre = dre["participant_id"]
df_nondre = nondre["participant_id"]

print(df_nondre)

349    sub-006
352    sub-010
353    sub-012
354    sub-013
355    sub-014
356    sub-015
358    sub-019
360    sub-021
362    sub-024
363    sub-025
365    sub-027
366    sub-029
367    sub-031
369    sub-035
Name: participant_id, dtype: object


In [187]:
x1 = dre["NSC_SalVentAttn"]
x2 = nondre["NSC_SalVentAttn"]

In [188]:
tstats,pvals,dfs = ttest_ind(x1, x2, alternative='two-sided', usevar='pooled', weights=(None, None), value=0)
pvals

0.3185015225457177

In [193]:
#get generalized seizure and nongeneralized groups
gen = df_LOBE[df_LOBE["generalized_seizures"] == 1]
nongen = df_LOBE[df_LOBE["generalized_seizures"] != 1]

#make sure pulling right subs
df_gen = gen["participant_id"]
df_nongen = nongen["participant_id"]

print(df_gen)

349    sub-006
353    sub-012
355    sub-014
357    sub-017
358    sub-019
359    sub-020
362    sub-024
364    sub-026
366    sub-029
368    sub-033
369    sub-035
Name: participant_id, dtype: object


In [238]:
x1 = gen["NFC_SomMot"]
x2 = nongen["NFC_SomMot"]

In [239]:
tstats,pvals,dfs = ttest_ind(x1, x2, alternative='two-sided', usevar='pooled', weights=(None, None), value=0)
pvals

0.7986519725951767

In [242]:
#get female and male groups
female = df_LOBE[df_LOBE["sex"] == "F"]
male = df_LOBE[df_LOBE["sex"] == "M"]

#make sure pulling right subs
df_female = female["participant_id"]
df_male = male["participant_id"]

print(df_male)

349    sub-006
351    sub-009
352    sub-010
355    sub-014
356    sub-015
362    sub-024
364    sub-026
365    sub-027
366    sub-029
367    sub-031
368    sub-033
369    sub-035
Name: participant_id, dtype: object


In [292]:
x1 = female["NFC_SalVentAttn"]
x2 = male["NFC_SalVentAttn"]

In [293]:
tstats,pvals,dfs = ttest_ind(x1, x2, alternative='two-sided', usevar='pooled', weights=(None, None), value=0)
pvals

0.34685893640229604

In [36]:
rejected,corr_pvals = fdrcorrection(pvals, alpha=0.05, method='indep', is_sorted=False)
rejectedparticipant_id 	dataset 	age 	sex 	age_seizure_onset 	age_ep_diagnosis 	ep_duration_first_scan 	seizure_duration_first_scan 	pnes 	benign_rolandic 	... 	NFC_Limbic 	NFC_Cont 	NFC_Default 	NSC_Vis 	NSC_SomMot 	NSC_DorsAttn 	NSC_SalVentAttn 	NSC_Limbic 	NSC_Cont 	NSC_Default
341 	sub-004 	LOBE 	NaN 	NaN 	NaN 	NaN 	NaN 	NaN 	NaN 	NaN 	... 	0.018775 	0.041168 	0.005356 	4.579539 	4.814665 	8.650834 	7.161025 	6.553801 	7.372455 	8.310777
342 	sub-006 	LOBE 	9.0 	M 	7.0 	8.0 	1.0 	2.0 	0.0 	0.0 	... 	0.242738 	0.337645 	0.289935 	4.145457 	4.446113 	7.507796 	7.520987 	6.103175 	7.077718 	7.506009
343 	sub-007 	LOBE 	9.0 	F 	7.0 	7.0 	2.0 	2.0 	0.0 	0.0 	... 	0.040016 	0.055915 	0.013688 	4.204648 	4.599075 	7.685924 	7.358690 	6.379661 	7.868797 	8.170206
344 	sub-009 	L

AssertionError: pvals must be 1-dimensional, that is of shape (n,)

In [5]:
tstats,pvals,dfs = ttest_ind(x1, x2, alternative='two-sided', usevar='pooled', weights=(None, None), value=0)
tstats.shape

,participant_id,dataset,age,sex,age_onset,ep_duration,pnes,focal_seizure_type,generalized_seizures,dre,...,NFC_Limbic,NFC_Cont,NFC_Default,NSC_Vis,NSC_SomMot,NSC_DorsAttn,NSC_SalVentAttn,NSC_Limbic,NSC_Cont,NSC_Default
0,sub-HCD0001305,HCP,11.916667,M,NaN,NaN,NaN,NaN,NaN,NaN,...,0.200019,0.257366,0.135539,3.885010,5.062987,8.027959,7.580041,5.651460,7.296828,7.935002
1,sub-HCD0021614,HCP,9.166667,F,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.000347,0.147511,0.039930,4.216869,4.518716,7.629381,7.669835,6.542762,7.380725,8.132475
2,sub-HCD0026119,HCP,15.166667,F,NaN,NaN,NaN,NaN,NaN,NaN,...,0.093052,0.082952,0.052142,4.770188,4.651514,8.898358,7.785734,6.827971,7.798882,8.109647
3,sub-HCD0041822,HCP,17.416667,M,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.001980,-0.008243,0.010302,4.030981,4.773803,8.804379,7.742067,6.960907,8.298339,8.540095
4,sub-HCD0042420,HCP,18.333333,M,NaN,NaN,NaN,NaN,NaN,NaN,...,0.091706,0.153798,0.082643,4.350406,4.851836,9.014367,7.819415,6.471102,8.094806,8.193476
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
367,sub-029,LOBE,16.000000,M,15.0,1.0,0.0,motor,1.0,0.0,...,0.467955,0.392260,0.372756,4.093281,4.358512,7.515517,7.141540,6.457969,6.372687,7.238880
368,sub-031,LOBE,9.000000,M,4.0,5.0,0.0,motor,0.0,NaN,...,0.228163,0.205326,0.162994,4.788529,5.042807,8.379897,7.876680,7.567836,8.409854,8.806705
369,sub-033,LOBE,7.000000,M,4.0,3.0,0.0,non-motor,1.0,1.0,...,-0.029804,0.006911,-0.027776,4.030132,4.791794,7.306059,7.469537,6.275113,7.390206,8.429542
370,sub-035,LOBE,7.000000,M,2.0,5.0,0.0,motor,1.0,0.0,...,0.035206,-0.010172,0.008450,4.027565,4.504583,7.768704,7.925532,6.047853,7.735365,8.119764


In [64]:
fc_pattern='results/{dataset}/sub-{subject}/func/sub-{subject}_task-rest_den-32k_desc-preproc_denoise-24HMP8PhysSpikeReg_fwhm-5_atlas-schaefer_bold.pconn.nii'
sc_pattern='results/{dataset}/sub-{subject}/dwi/sub-{subject}_den-32k_atlas-schaefer_struc.pconn.nii'
sfc_pattern='results/{dataset}/sub-{subject}/func/sub-{subject}_task-rest_den-32k_desc-preproc_denoise-24HMP8PhysSpikeReg_fwhm-5_atlas-schaefer_sfc.pscalar.nii'


datasets=['HCP','LOBE']



In [66]:
sfc_paths={}
sfc_paths['HCP'] = sorted(glob(sfc_pattern.format(dataset='HCP',subject='*')))
sfc_paths['LOBE'] = sorted(glob(sfc_pattern.format(dataset='LOBE',subject='*')))

fc_paths={}
fc_paths['HCP'] = sorted(glob(fc_pattern.format(dataset='HCP',subject='*')))
fc_paths['LOBE'] = sorted(glob(fc_pattern.format(dataset='LOBE',subject='*')))

sc_paths={}
sc_paths['HCP'] = sorted(glob(sc_pattern.format(dataset='HCP',subject='*')))
sc_paths['LOBE'] = sorted(glob(sc_pattern.format(dataset='LOBE',subject='*')))

#print(sfc_paths)
#print(fc_paths)
#print(sc_paths)

In [ ]:
sfc={}
fc={}
sc={}
num_parcels = nib.load(sfc_paths['LOBE'][0]).get_fdata().shape[1]

for dataset in datasets:
    sfc[dataset] = np.concatenate(  [ nib.load(path).get_fdata()  for path in sfc_paths[dataset]] ,axis=0)
    fc[dataset] = np.concatenate(  [ nib.load(path).get_fdata().reshape((1,num_parcels,num_parcels))  for path in fc_paths[dataset]] ,axis=0)
    sc[dataset] = np.concatenate(  [ nib.load(path).get_fdata().reshape((1,num_parcels,num_parcels))  for path in sc_paths[dataset]] ,axis=0)

In [ ]:
##SFC T-test and FDR - LOBE vs HCP

In [ ]:
sfc['LOBE'].shape

In [28]:
x1 = sfc['HCP']
x2 = sfc['LOBE']

In [29]:
x2.shape

(14, 300)

In [30]:
tstats,pvals,dfs = ttest_ind(x1, x2, alternative='two-sided', usevar='pooled', weights=(None, None), value=0)
tstats.shape

(300,)

In [31]:
pvals

array([1.70699709e-01, 1.78824864e-01, 2.26960136e-01, 1.88559300e-01,
       5.66890299e-01, 6.14631760e-01, 2.51952273e-01, 6.33617811e-01,
       4.13519648e-01, 7.38789719e-01, 8.39075179e-01, 1.03768428e-01,
       1.49555553e-01, 7.08002286e-01, 3.33001362e-01, 7.75473839e-01,
       2.85164578e-01, 2.14727308e-01, 5.78901077e-01, 5.89053450e-01,
       6.43934225e-02, 7.63964296e-02, 8.08575307e-03, 3.11774055e-04,
       8.87974178e-01, 7.75437313e-01, 9.24945788e-01, 2.60681315e-01,
       4.10744700e-01, 1.53628551e-01, 5.92039347e-01, 1.62312334e-01,
       8.60003899e-01, 4.21485002e-02, 4.13560234e-01, 8.06349544e-02,
       1.08980301e-02, 2.69804073e-01, 1.11270406e-01, 3.41460786e-02,
       6.03697259e-01, 5.46079259e-01, 4.87668753e-01, 1.27429067e-01,
       1.33117458e-01, 5.25288761e-01, 3.95303180e-01, 3.23160883e-01,
       3.27658719e-02, 1.02608151e-01, 3.16066632e-01, 5.67082663e-01,
       7.53478841e-01, 8.17526421e-01, 8.78900322e-01, 4.71310760e-01,
      

In [32]:
rejected,corr_pvals = fdrcorrection(pvals, alpha=0.05, method='indep', is_sorted=False)

In [33]:
rejected

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False,  True, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False,  True, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,

In [56]:
# FC avg per parcel, group difference
#set diagonal to be nan
x1 = fc['HCP']
for s in range(x1.shape[0]):
    conn = x1[s,:,:]    
    mask = np.eye(num_parcels)
    conn[mask==1] = np.nan
    x1[s,:,:] = conn

x2 = fc['LOBE']
for s in range(x2.shape[0]):
    conn = x2[s,:,:]
    mask = np.eye(num_parcels)
    conn[mask==1] = np.nan
    x2[s,:,:] = conn

In [57]:
x1.shape

(341, 300, 300)

In [58]:
x1 = np.nanmean(x1,axis=2)
x2 = np.nanmean(x2,axis=2)

In [59]:
x1.shape

(341, 300)

In [60]:
x2.shape

(14, 300)

In [37]:
tstats,pvals,dfs = ttest_ind(x1, x2, alternative='two-sided', usevar='pooled', weights=(None, None), value=0)
tstats.shape

(300,)

In [38]:
rejected,corr_pvals = fdrcorrection(pvals, alpha=0.05, method='indep', is_sorted=False)

In [39]:
rejected

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,

In [40]:
# SC avg per parcel, group difference
#set diagonal to be nan
x1 = sc['HCP']
for s in range(x1.shape[0]):
    conn = x1[s,:,:]    
    mask = np.eye(num_parcels)
    conn[mask==1] = np.nan
    x1[s,:,:] = conn

x2 = sc['LOBE']
for s in range(x2.shape[0]):
    conn = x2[s,:,:]
    mask = np.eye(num_parcels)
    conn[mask==1] = np.nan
    x2[s,:,:] = conn

In [41]:
x1.shape

(340, 300, 300)

In [42]:
x1 = np.nanmean(x1,axis=2)
x1.shape
x2 = np.nanmean(x2,axis=2)
x2.shape

(14, 300)

In [61]:
x1.shape

(341, 300)

In [62]:
x2.shape

(14, 300)

In [43]:
tstats,pvals,dfs = ttest_ind(x1, x2, alternative='two-sided', usevar='pooled', weights=(None, None), value=0)
tstats.shape

(300,)

In [44]:
rejected,corr_pvals = fdrcorrection(pvals, alpha=0.05, method='indep', is_sorted=False)

In [45]:
rejected

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,